In [1]:
pip install hvplot


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 6.3 MB/s eta 0:00:00


In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container {width:100% !important; }</style>"))

import glob
import pandas as pd
import numpy as np
import hvplot.pandas
import plotly.express as px
import plotly.graph_objects as go
pd.options.mode.chained_assignment = None

import datetime as dt
%autosave 30

Autosaving every 30 seconds


In [3]:
CPU_FREQ=2.60

rdtsc_df_dict = []
ttt_df_dict = []
for filename in glob.glob("./kse*.log"):
    print('processing {}'.format(filename))
    for line in open(filename):
        tokens = line.strip().split(' ')
        if len(tokens) != 4:
            continue

        try:
            time = tokens[0]
            tag = tokens[2]
            latency = float(tokens[3])
            latency_rdtsc = latency / CPU_FREQ
            time_datetime = pd.to_datetime(time, format='%H:%M:%S.%f')
        except:
            continue

        if ' RDTSC ' in line:
            if tokens[1] != 'RDTSC':
                continue

            rdtsc_df_dict.append({'timestamp':time, 'tag':tag, 'latency':latency_rdtsc})
        elif ' TTT ' in line:
            if tokens[1] != 'TTT':
                continue

            ttt_df_dict.append({'timestamp':time, 'tag':tag, 'latency':latency})

rdtsc_df = pd.DataFrame.from_dict(rdtsc_df_dict)
rdtsc_df = rdtsc_df.drop_duplicates().sort_values(by='timestamp')
rdtsc_df['timestamp'] = pd.to_datetime(rdtsc_df['timestamp'], format='%H:%M:%S.%f')

ttt_df = pd.DataFrame.from_dict(ttt_df_dict)
ttt_df = ttt_df.drop_duplicates().sort_values(by='timestamp')
ttt_df['timestamp'] = pd.to_datetime(ttt_df['timestamp'], format='%H:%M:%S.%f')

processing ./kse_matching_engine.log
processing ./kse_order_server.log
processing ./kse_order_server_responses.log
processing ./kse_market_data_publisher.log


In [4]:
for tag in rdtsc_df['tag'].unique():
    print(tag)

    fig = go.Figure()

    t_df = rdtsc_df[rdtsc_df['tag'] == tag].copy()
    t_df = t_df[t_df['latency'] > 0]

    q_hi = t_df['latency'].quantile(0.99)
    q_lo = t_df['latency'].quantile(0.01)
    t_df = t_df[(t_df['latency'] < q_hi) & (t_df['latency'] > q_lo)]

    mean = t_df['latency'].astype(float).mean()
    print('{} has {} observations mean {}'.format(tag, len(t_df), mean))

    rolling_window = max(1, int(len(t_df) / 100))

    use_micros = False
    if mean >= 1000:
        use_micros = True
        t_df['latency'] = t_df['latency'].astype(float) / 1000

    fig.add_trace(go.Scatter(x=t_df['timestamp'], y=t_df['latency'], name=tag))
    fig.add_trace(go.Scatter(x=t_df['timestamp'], y=t_df['latency'].rolling(rolling_window).mean(), name=tag + ' mean'))
#     fig.add_trace(go.Scatter(x=t_df['timestamp'], y=t_df['latency'].rolling(rolling_window).std(), name=tag + ' std'))

    fig.update_layout(title='performance ' + tag + ' ' + ('microseconds' if use_micros else 'nanoseconds'), height=750, width=1000, hovermode='x', legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ))
    fig.show()

Exchange_odsSerialization
Exchange_odsSerialization has 2005 observations mean 406.58123920966807


Exchange_odsDeserialization
Exchange_odsDeserialization has 1096 observations mean 26.844469399213924


Exchange_FIFOSequencer_addClientRequest
Exchange_FIFOSequencer_addClientRequest has 1088 observations mean 125.14493778280543


Exchange_FIFOSequencer_sequenceAndPublish
Exchange_FIFOSequencer_sequenceAndPublish has 997 observations mean 20185.47025692462


Exchange_MEOrderBook_checkForMatch
Exchange_MEOrderBook_checkForMatch has 911 observations mean 39230.85198007261


...

Exchange_MatchingEngine_processClientRequest
Exchange_MatchingEngine_processClientRequest has 1097 observations mean 88028.2722109249


Exchange_mdpubSerialization
Exchange_mdpubSerialization has 1779 observations mean 96.02412764301465


Exchange_MEOrderBook_removeOrder
Exchange_MEOrderBook_removeOrder has 434 observations mean 462.7100319035803


Exchange_MEOrderBook_cancel
Exchange_MEOrderBook_cancel has 186 observations mean 59247.390405293634


Exchange_MEOrderBook_match
Exchange_MEOrderBook_match has 452 observations mean 76952.73144996596


In [5]:
HOPS = [
    ['T1_OrderServer_TCP_read', 'T2_OrderServer_LFQueue_write'],
    ['T2_OrderServer_LFQueue_write', 'T3_MatchingEngine_LFQueue_read'],
    ['T3_MatchingEngine_LFQueue_read', 'T4_MatchingEngine_LFQueue_write'], ['T3_MatchingEngine_LFQueue_read', 'T4t_MatchingEngine_LFQueue_write'],
    ['T4_MatchingEngine_LFQueue_write', 'T5_MarketDataPublisher_LFQueue_read'], ['T4t_MatchingEngine_LFQueue_write', 'T5t_OrderServer_LFQueue_read'],
    ['T5_MarketDataPublisher_LFQueue_read', 'T6_MarketDataPublisher_UDP_write'], ['T5t_OrderServer_LFQueue_read', 'T6t_OrderServer_TCP_write'], ['T1_OrderServer_TCP_read', 'T6t_OrderServer_TCP_write'], ['T1_OrderServer_TCP_read', 'T6_MarketDataPublisher_UDP_write']
]

In [6]:
for tags in HOPS:
    tag_p, tag_n = tags
    print('{} => {}. {} => {}.'.format(tag_p, len(ttt_df[ttt_df['tag'] == tag_p]), tag_n, len(ttt_df[ttt_df['tag'] == tag_n])))

    fig = go.Figure()

    t_df = ttt_df[(ttt_df['tag'] == tag_n) | (ttt_df['tag'] == tag_p)]
    t_df['latency_diff'] = t_df['latency'].diff()
    t_df = t_df[t_df['latency_diff'] > 0]
    t_df = t_df[t_df.tag == tag_n]

    q_hi = t_df['latency_diff'].quantile(0.99)
    q_lo = t_df['latency_diff'].quantile(0.01)
    t_df = t_df[(t_df['latency_diff'] < q_hi) & (t_df['latency_diff'] > q_lo)]

    mean = t_df['latency_diff'].astype(float).mean()
    print('{} has {} observations mean {}'.format(tag_n, len(t_df), mean))

    rolling_window = max(1, int(len(t_df) / 100))

    unit = 'nanoseconds'
    if mean >= 1000000:
        unit = 'milliseconds'
        t_df['latency_diff'] = t_df['latency_diff'].astype(float) / 1000000
    elif mean >= 1000:
        unit = 'microseconds'
        t_df['latency_diff'] = t_df['latency_diff'].astype(float) / 1000

    tag_name = tag_p + ' -> ' + tag_n
    fig.add_trace(go.Scatter(x=t_df['timestamp'], y=t_df['latency_diff'], name=tag_name))
    fig.add_trace(go.Scatter(x=t_df['timestamp'], y=t_df['latency_diff'].rolling(rolling_window).mean(), name=tag_name + ' mean'))

    fig.update_layout(title='performance ' + tag_name + ' ' + unit, height=750, width=1000, hovermode='x', legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ))
    fig.show()


T1_OrderServer_TCP_read => 1085. T2_OrderServer_LFQueue_write => 1121.
T2_OrderServer_LFQueue_write has 1095 observations mean 82574.61187214612


T2_OrderServer_LFQueue_write => 1121. T3_MatchingEngine_LFQueue_read => 1121.
T3_MatchingEngine_LFQueue_read has 838 observations mean 108428.52505966587


T3_MatchingEngine_LFQueue_read => 1121. T4_MatchingEngine_LFQueue_write => 1833.
T4_MatchingEngine_LFQueue_write has 1795 observations mean 66871.62116991644


T3_MatchingEngine_LFQueue_read => 1121. T4t_MatchingEngine_LFQueue_write => 2045.
T4t_MatchingEngine_LFQueue_write has 2003 observations mean 41319.5247129306


T4_MatchingEngine_LFQueue_write => 1833. T5_MarketDataPublisher_LFQueue_read => 1833.
T5_MarketDataPublisher_LFQueue_read has 1786 observations mean 97537.72004479283


T4t_MatchingEngine_LFQueue_write => 2045. T5t_OrderServer_LFQueue_read => 2052.
T5t_OrderServer_LFQueue_read has 1307 observations mean 6626.228003060443


T5_MarketDataPublisher_LFQueue_read => 1833. T6_MarketDataPublisher_UDP_write => 1065.
T6_MarketDataPublisher_UDP_write has 1043 observations mean 615417.3729626079


T5t_OrderServer_LFQueue_read => 2052. T6t_OrderServer_TCP_write => 2052.
T6t_OrderServer_TCP_write has 2007 observations mean 114799.73692077729


T1_OrderServer_TCP_read => 1085. T6t_OrderServer_TCP_write => 2052.
T6t_OrderServer_TCP_write has 2001 observations mean 214803.06246876562


T1_OrderServer_TCP_read => 1085. T6_MarketDataPublisher_UDP_write => 1065.
T6_MarketDataPublisher_UDP_write has 1043 observations mean 704829.8523489933
